In [1]:
!date

Fri Sep 29 17:44:46 EDT 2023


# Grab lake contours bigger than 5 degrees - no time tracking
## Make distance and direction field, lake by lake (time by time)
#### from all available datafiles, their names end in distance.nc

------------


In [41]:
from shapely import geometry
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import fiona             # a read-write library for shapefiles
import os         
#from descartes.patch import PolygonPatch

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

from glob import glob
import xarray as xr
#import dask.array as da

import pandas as pd
import geopandas as gp
from datetime import datetime

In [42]:
files = glob('/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/MERRA2_2D/MINIMAL/*distance.nc')

In [43]:
# CRS warnings are annoying below, I might suppress all just for readability

import warnings
warnings.filterwarnings("ignore")

# A function to return a GeoDataFrame of polygons 

In [44]:
# loop over contour collections (and polygons in each collection)
# store in polylist  
def gdf_from_contours(lon,lat,tqv,conlevel):
    
    levels = [conlevel, 9e9]
    cs = plt.contourf(lon,lat,tqv,levels)
# create lookup table for levels
    lvl_lookup = dict(zip(cs.collections, cs.levels))
    
    zvalues, polylist  = [], []
#    i=0
    for col in cs.collections:
        z=lvl_lookup[col] # the value of this level
        for contour_path in col.get_paths():
#        print('contour path: ',i); i = i+1
        # create the polygon for this level
            for ncp,cp in enumerate(contour_path.to_polygons()):
#            print('   ncp: ', ncp)
                lons = np.array(cp)[:,0]
                lats = np.array(cp)[:,1]
                new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(lons,lats)])            
                if ncp == 0:
                    poly = new_shape # first shape
                else:
                    poly = poly.difference(new_shape) # Remove the holes

            polylist.append(poly)
            zvalues.append(z)
        
        gdf = gp.GeoDataFrame(geometry=polylist)
        gdf['tqv_values']=zvalues
        gdf['perimeter']=gdf.length
        gdf['area']=gdf.area
        gdf['centroidlat']=gdf.centroid.y
        gdf['centroidlon']=gdf.centroid.x
        gdf['centriod_is_inside']= gdf.contains(gdf.centroid)
        gdf['maxlon']=gdf.bounds.maxx
        gdf['minlon']=gdf.bounds.minx
        gdf['maxlat']=gdf.bounds.maxy
        gdf['minlat']=gdf.bounds.miny

        return(gdf)

------------

# Open each dataset in xarray, and loop over all times in it

In [45]:
MINSIZE = 5 # square degrees, smallest area lake (closed contour at an instant) to process 

In [ ]:
all_gdflakes = [] # collect them all in one big index set 

for ifile in range( len(files) ):  
    file = files[ifile] # replace with a loop to do all the MERRA2_MINIMAL data files
    print(file)
    bigds = xr.open_dataset(file)
    
    # Make an array of Points that are the gridpoints of MERRA2 in this file
    lat2d = bigds.lat.values[:,None]   + bigds.lon.values*0
    lon2d = bigds.lat.values[:,None]*0 + bigds.lon.values
    points = gp.GeoSeries( [Point(x, y) for x, y in zip(lon2d.ravel(),lat2d.ravel()) ] )\
            .set_crs(epsg = "4326", inplace = True)
     
# TIME LOOP OVER ALL TIMES: SLICE OFF ds DATASET - 2D ONLY
    for itime in range(len(bigds.time)): 
        ds = bigds.isel(time=itime).drop_vars(['distance','dir_from_centroid'])
        gdf = gdf_from_contours(ds.lon,ds.lat,ds.tqv, 55.)
    
        # Screen for "LAKES" inbounds and big enough (>5 square degrees) 
        inbounds = \
            (gdf.minlon > ds.lon.min().values) & (gdf.maxlon < ds.lon.max().values) & \
            (gdf.minlat > ds.lat.min().values) & (gdf.maxlat < ds.lat.max().values)
        gdflakes = gdf[ (inbounds == True) & (gdf.area > MINSIZE) ].copy().reindex()
    
# Now loop over all those lakes, making a distance field for each lake
        i=0 # counter for lakes at this time
        for lake in gdflakes.itertuples():
# Compute distance field from perimiter:
            dist = points.distance(lake.geometry.boundary).values.reshape(len(ds.lat),len(ds.lon))
            isin = points.within(lake.geometry).values.reshape(len(ds.lat),len(ds.lon))
            dist *= (-2)*(isin-0.5)  # make SIGNED distance from boundary, positive is exterior 

# Direction to centroid of WHOLE LAKE at this time
            centlon = lake.centroidlon
            centlat = lake.centroidlat 
# use j in explicit loop over gridpoints, since geodesic takes scalar only. Inelegant but works. 
            dir_to = []
            for j in range(len(lon2d.ravel())): 
                fwd_az,back_az,d = geodesic.inv(lon2d.ravel()[j], lat2d.ravel()[j], [centlon], [centlat])
                dir_to.append(fwd_az+180)
                
            dist_to = np.array(dist).reshape(len(ds.lat),len(ds.lon))
            dir_to = np.array(dir_to).reshape(len(ds.lat),len(ds.lon))

            command = "ds = ds.assign(distance"+str(i)+"=(['lat','lon'],dist))"
            exec(command)
            command = "ds = ds.assign(dir_to"+str(i)+"=(['lat','lon'],dir_to))"
            exec(command)
            i += 1 
            
            # write geojson of lakes at this time into one file -- no need really 
            # keeping all_gdflakes so that can be used later 
            #with open(file[:-2]+str(itime)+'lakes_contours.geojson', 'w') as f:
            #    f.write(gdflakes.to_json())
            #    f.close()
            
## SHOULD REALLY PUT THESE IN FOLDERS, BUT EASY IS JUST MAKE A FILENAME FOR NOW TO SEE IF IT WORKS 
            ds.to_netcdf(file[:-2]+str(itime)+'_distdirs.nc')
            ds.close()
                     
        all_gdflakes.append(gdflakes) # giant list of all lakes at all times in all files 

In [ ]:
all_gdflakes